# <b><span style='color:#2563eb'>00 | </span>Modele predykcji</b>

## <span style='color:#2563eb'>🔷 | <b></span>Import bibliotek</b>

In [ ]:
# Set auto reload after making changes
%load_ext autoreload
%autoreload 2

import os
import json
import numpy as np
import pandas as pd
from datetime import timedelta

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

# Modeling
import xgboost as xgb
import torch

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

# Wrote myself
from source.CustomPlot import CustomPlot
from source.Utils import SplitDateColumn, AddPrefixToColumns, DescribeData
from source.EvaluateModel import EvaluteModel

# GPU or CPU use for model
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

## <span style='color:#2563eb'>🔷 | <b></span>Zbiory</b>

In [ ]:
PATH = 'data/'

gas_prices = pd.read_csv(os.path.join(PATH, 'gas_prices.csv'),
                   dtype={'lowest_price_per_mwh': 'float64',
                          'highest_price_per_mwh': 'float64',
                          'data_block_id': 'int64'},
                   parse_dates=['forecast_date', 'origin_date'])

electricity_prices = pd.read_csv(os.path.join(PATH, 'electricity_prices.csv'),
                   dtype={'euros_per_mwh': 'float64',
                          'data_block_id': 'int64'},
                   parse_dates=['forecast_date', 'origin_date'])

historical_weather = pd.read_csv(os.path.join(PATH, 'historical_weather.csv'),
                dtype={'temperature': 'float64',
                        'dewpoint': 'float64',
                        'rain': 'float64',
                        'snowfall': 'float64',
                        'surface_pressure': 'float64',
                        'cloudcover_total': 'int16',
                        'cloudcover_low': 'int16',
                        'cloudcover_mid': 'int16',
                        'cloudcover_high': 'int16',
                        'winddirection_10m': 'int16',
                        'shortwave_radiation': 'float64',
                        'direct_solar_radiation' : 'float64',
                        'diffuse_radiation': 'float64',
                        'latitude': 'float64',
                        'longitude' : 'float64',
                        'data_block_id' : 'int64'},

                parse_dates=['datetime'])

forecast_weather = pd.read_csv(os.path.join(PATH, 'forecast_weather.csv'),
                dtype={'temperature': 'float64',
                        'dewpoint': 'float64',
                        'total_precipitation': 'float64',
                        'snowfall': 'float64',
                        'cloudcover_total': 'float64',
                        'cloudcover_low': 'float64',
                        'cloudcover_mid': 'float64',
                        'cloudcover_high': 'float64',
                        '10_metre_u_wind_component': 'float64',
                        '10_metre_v_wind_component': 'float64',
                        'direct_solar_radiation' : 'float64',
                        'surface_solar_radiation_downwards': 'float64',
                        'latitude': 'float64',
                        'longitude' : 'float64',
                        'data_block_id' : 'int64',
                        'hours_ahead': 'int16'},

                parse_dates=['origin_datetime', 'forecast_datetime'])

train = pd.read_csv(os.path.join(PATH, 'train.csv'),
                dtype={ 'county': 'int16',
                        'is_business': 'boolean',
                        'product_type': 'int8',
                        'target': 'float64',
                        'is_consumption': 'boolean',
                        'data_block_id' : 'int64',
                        'row_id' : 'int16',
                        'prediction_unit_id' : 'int16' },

                parse_dates=['datetime'])

client = pd.read_csv(os.path.join(PATH, 'client.csv'),
                dtype={ 'county': 'int16',
                        'is_business': 'boolean',
                        'product_type': 'int8',
                        'eic_count': 'float64',
                        'installed_capacity': 'float64',
                        'data_block_id' : 'int64'},

                parse_dates=['date'])

In [ ]:
weather_station = pd.read_csv(os.path.join(PATH, 'weather_station_to_county_mapping.csv'),
                   dtype={'county_name': 'str',
                          'longitude': 'float64',
                          'latitude': 'float64',
                          'county': 'float64'})

weather_station.dropna(subset='county', inplace=True)
weather_station.drop(columns=['county_name'], inplace=True)
weather_station['county'] = weather_station['county'].astype('int')
weather_station[['latitude', 'longitude']] = weather_station[['latitude', 'longitude']].astype(float).round(1)

## <span style='color:#2563eb'>🔷 | <b></span>Scelenie zbiorów</b>

In [ ]:
from source.Utils import SplitDateColumn, AddPrefixToColumns

class FeaturesProcessing():
    def __init__(self,
            train: pd.DataFrame,
            client: pd.DataFrame,
            gas_prices: pd.DataFrame,
            forecast_weather: pd.DataFrame,
            electricity_prices: pd.DataFrame,
            weather_station: pd.DataFrame):
        
        self.train = train
        self.weather_station = weather_station

        self.client = self.__prepareClient(client)
        self.gas_prices = self.__prepareGasPrices(gas_prices)
        self.forecast_weather = self.__prepareForecastData(forecast_weather)
        self.electricity_prices = self.__prepareEnergyPrices(electricity_prices)

        self.data = train.copy()
        self.__merge('gas prices', self.gas_prices, on=['data_block_id'])
        self.__merge('electricity prices', self.electricity_prices, on=['datetime', 'data_block_id'])
        self.__merge('client', self.client, on=['county', 'is_business', 'product_type', 'data_block_id'])
        self.__merge('forecast weather', self.forecast_weather, on=['datetime', 'county', 'data_block_id'])
        
        self.__AddCustomFeatures()

        SplitDateColumn(self.data, 'datetime')

    def getData(self, dropNa = False):
        if dropNa:
            return self.data.dropna()
        
        return self.data

    def __merge(self, datasetName: str, data: pd.DataFrame, on=[], how='left') -> pd.DataFrame:
        print(f'MERGING: <- {datasetName}')
        print(f'- Before: <- {len(self.data)} rows')

        self.data = self.data.merge(data, how=how, on=on)
        print(f'- After: {len(self.data)} rows')
        print()

    def __prepareClient(self, client: pd.DataFrame) -> pd.DataFrame:
        client = client.drop(columns=['date'])

        return client

    def __prepareForecastData(self, forecast_weather: pd.DataFrame) -> pd.DataFrame:
        forecast_weather = forecast_weather.rename(columns = {'forecast_datetime': 'datetime'})
        forecast_weather.drop(columns = 'origin_datetime', inplace=True)
        forecast_weather['datetime'] = forecast_weather['datetime'].dt.tz_convert('Europe/Brussels').dt.tz_localize(None)

        # Map to weather locations
        forecast_weather[['latitude', 'longitude']] = forecast_weather[['latitude', 'longitude']].astype(float).round(1)
        forecast_weather = forecast_weather.merge(self.weather_station, how='left', on=['latitude', 'longitude'])

        # Some weather locations are outside any county
        forecast_weather.dropna(subset='county', inplace=True)

        forecast_weather['county'] = forecast_weather['county'].astype(int)

        # Some county have many weather locations
        forecast_weather = forecast_weather.groupby(by=['datetime', 'county', 'data_block_id']).mean().reset_index()
        
        return forecast_weather

    def __prepareEnergyPrices(self, electricity_prices: pd.DataFrame) -> pd.DataFrame:
        columns = ['euros_per_mwh', 'data_block_id']

        ep = electricity_prices[columns].copy()
        ep['datetime'] = electricity_prices['forecast_date'] + timedelta(days=1)

        AddPrefixToColumns(ep, ['euros_per_mwh'], 'elec_price_')

        return ep
    
    def __prepareGasPrices(self, gas_prices: pd.DataFrame) -> pd.DataFrame:
        columns = ['highest_price_per_mwh', 'lowest_price_per_mwh', 'data_block_id']

        gp = gas_prices[columns].copy()

        AddPrefixToColumns(gp, ['highest_price_per_mwh', 'lowest_price_per_mwh'], 'gas_')
        
        return gp
    
    def __AddCustomFeatures(self):
        merge = ['county', 'is_business', 'product_type', 'data_block_id', 'is_consumption', 'datetime']

        feature = 'target_week_ago'
        trainMinus7 = self.train.copy()
        trainMinus7['datetime'] = trainMinus7['datetime'] + timedelta(days=7)
        trainMinus7['data_block_id'] = trainMinus7['data_block_id'] + 7
        trainMinus7.rename(columns={'target' : feature}, inplace=True)

        self.data = self.data.merge(trainMinus7[merge + [feature]], how='left', on=merge)

        feature = 'target_3_days_ago'
        trainMinus3 = self.train.copy()
        trainMinus3['datetime'] = trainMinus3['datetime'] + timedelta(days=3)
        trainMinus3['data_block_id'] = trainMinus3['data_block_id'] + 3
        trainMinus3.rename(columns={'target' : feature}, inplace=True)

        self.data = self.data.merge(trainMinus3[merge + [feature]], how='left', on=merge)

In [ ]:
fp = FeaturesProcessing(
    train=train,
    client= client,
    gas_prices=gas_prices,
    forecast_weather=forecast_weather,
    electricity_prices=electricity_prices,
    weather_station=weather_station
    )

In [ ]:
DescribeData(fp.getData())

## <span style='color:#2563eb'>🔷 | <b></span>Podział na zbiory do modelów</b>

In [ ]:
features = [
    'is_business',
    'product_type',
    'is_consumption',
    'county',
    'temperature',
    'dewpoint',
    'cloudcover_high',
    'cloudcover_low',
    'cloudcover_mid',
    'cloudcover_total',
    '10_metre_u_wind_component',
    '10_metre_v_wind_component',
    'direct_solar_radiation',
    'surface_solar_radiation_downwards',
    'snowfall',
    'total_precipitation',
    'installed_capacity',
    'elec_price_euros_per_mwh',
    'datetime_hour',
    'gas_highest_price_per_mwh',
    'gas_lowest_price_per_mwh',
    'target_week_ago',
    'target_3_days_ago'
    ]
target_columns = ['target']

mergedData = fp.getData(dropNa=True)

X = mergedData[features]
y = mergedData[target_columns]

# Splitting into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### ✨ <b>Ocena modelu</b>

In [ ]:
county_name = json.load(open(os.path.join(PATH, 'county_id_to_name_map.json')))
county_name = {int(k):str(v) for k,v in county_name.items()}

modelEvaluator = EvaluteModel(X_test, y_test, mergedData, features, county_name)

## <span style='color:#2563eb'>🔷 | <b></span>XGBoost</b>

In [ ]:
clf = xgb.XGBRegressor (
    # Device ordinal, available options are cpu, cuda, and gpu.
    device = device, 
    enable_categorical=True,
    # Number of gradient boosted trees
    n_estimators = 1000,
    # Step size shrinkage used in update to prevents overfitting
    eta=0.1,
    # Activates early stopping. Validation metric needs to improve at least once in every early_stopping_rounds round(s) to continue training
    early_stopping_rounds=100,
    # L2 regularization term on weights. Increasing this value will make model more conservative
    reg_lambda = 1,
    # L1 regularization term on weights. Increasing this value will make model more conservative
    reg_alpha = 0,
    # Minimum loss reduction required to make a further partition on a leaf node of the tree
    gamma = 0,
    # Specify the learning task and the corresponding learning objective
    objective = 'reg:absoluteerror'
)

### ✨ <b>Uczenie</b>

In [ ]:
clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=True)

In [ ]:
modelEvaluator.test(clf, random_day=True)

In [ ]:
clf.predict(X_test)

### ✨ <b>Wyniki</b>

In [ ]:
pd.DataFrame({'name': clf.feature_names_in_, 'importance': clf.feature_importances_}).sort_values(by='importance', ascending=False)

## <span style='color:#2563eb'>🔷 | <b></span>Las losowy</b>

In [ ]:
clf = RandomForestRegressor(max_depth=10, n_estimators=5)

### ✨ <b>Uczenie</b>

In [ ]:
clf.fit(X_train, y_train)

### ✨ <b>Wyniki</b>

In [ ]:
y_pred = clf.predict(X_test)

np.sqrt(MSE(y_test, y_pred)) 

## <span style='color:#2563eb'>🔷 | <b></span>Szukanie hiperparametrów</b>

In [ ]:
model = xgb.XGBRegressor (
    # Device ordinal, available options are cpu, cuda, and gpu.
    device = device, 
    enable_categorical=True,
    # Number of gradient boosted trees
    n_estimators = 1000,
    # Step size shrinkage used in update to prevents overfitting
    eta=0.1,
    # Activates early stopping. Validation metric needs to improve at least once in every early_stopping_rounds round(s) to continue training
    early_stopping_rounds=100,
    # L2 regularization term on weights. Increasing this value will make model more conservative
    reg_lambda = 1,
    # L1 regularization term on weights. Increasing this value will make model more conservative
    reg_alpha = 0,
    # Minimum loss reduction required to make a further partition on a leaf node of the tree
    gamma = 0,
 )

# A parameter grid for XGBoost
params = {
        'n_estimators': [1000],
        'eta': [0.1, 0.3, 0.5],
        'reg_lambda': [0.5, 1, 2],
        }

In [ ]:
#clf = GridSearchCV(model, params, verbose=2)

#clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=False)

## <span style='color:#2563eb'>🔷 | <b></span>Załączanie odpowiedzi</b>

In [ ]:
import enefit
env = enefit.make_env()
iter_test = env.iter_test()

for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:

      data = FeaturesProcessing(
              train = test,
              client = client,
              forecast_weather = forecast_weather,
              gas_prices = gas_prices,
              electricity_prices = electricity_prices,
              weather_station = weather_station).getData()

      sample_prediction['target'] = clf.predict(data)
      env.predict(sample_prediction)